# Stochastic Optimisation Project 2025
by Timur Kharisov, Redouane Yagouti and William Ford.


### Problem description

In this project we solve logistic regression with L2 regularisation for the MNIST dataset. The data $x \in \mathbb{R}^{784} $ are 60,000 $28 \times 28$ greyscale images of digits from $0$-$9$, of which 12,000 were used for validation, and a further 10,000 further for testing. These are labelled as binary unit vectors $y\mathbb{R}^{10}$ with $y_i = 1$ if the image represents the digit $i$, else $y_j =0$.


The model parameters are a weights $W \in \R^{10 \times 784}$ (we use no biases), and logistic cost for a pair $(x, y)$
$$
f_{W} : \mathbb{R}^{784} \to \mathbb{R}
$$
$$
(x, y) \mapsto \log\left( \sum_{j=1}^{9} \exp((Wx)_j)\right) - \langle y, Wx\rangle_{\mathbb{R}^{10}}
$$
Assuming the law of $(x, y) \sim \mathcal{D}$ is unknown the objective is
$$
F_\alpha(W) = \mathbb{E}_{(x, y) \sim \mathcal{D}}[f_{W}(x, y)] + \frac{\alpha}{2} \lVert W \rVert^2
$$
where $\alpha$ is a regularisation hyperparameter which we would like to choose optimally. We would like to solve
$$
\min_{W} F_\alpha(W).
$$
By the law of large numbers , we approximate the expectation with the empirical mean over the training data and solve the problem
$$
\underset{W}{\operatorname{Argmin}} \; \hat{F}_\alpha(W) := \frac{1}{48000} \sum_{i=1}^{48000} f_{W}(x_i^{\text{train}}, y_i^{\text{train}}) + \frac{\alpha}{2} \lVert W \rVert^2
$$
This amounts to a convex optimisation problem in $\mathbb{R}^{7840}$, which is further strongly $\alpha$-convex from the regularisation term.

Searching for the optimal $\alpha$, we consider an accuracy of average success of parameters $W$;
$$
L(W) = \frac{1}{12000} \sum_{i=1}^{12000} l(W, x_i^{\text{valid}}, y_i^{\text{valid}})
$$
where
$$
l(x, y) = \begin{cases} 1 & Wx \text{ and } y \text{ have the same largest coordinate}\\
0 & \text{ otherwise}. \end{cases}
$$
We then want to identify
$$
\underset{\alpha}{\operatorname{Argmax}}\left\{ L(W_\alpha) : W_\alpha = \underset{W}{\operatorname{Argmin}} \; \hat{F}_\alpha(W)\right\}.
$$

### Analysis of the Objective

We compute the gradient of $f( \cdot, x, y)$ with respect to a weight $w_{kl}$:
$$
\frac{\partial}{\partial w_{kl}} \left[ \log \left( \sum_{j=0}^9 e^{(Wx)_j}\right) - \langle y, W x \rangle_{\mathbb{R}^{10}} \right] = \frac{e^{(Wx)_k}}{\sum_{j=0}^9 e^{(Wx)_j}} \cdot  x_l - y_k x_l.
$$
Noting that the second term is linear in $W$, it is sufficient to deduce the monotonicity of the first term in the sense $\langle \nabla_W f(W_1, x, y) - \nabla f(W_0, x, y) | W_1 - W_0 \rangle_{\mathbb{R}^{7840}} \geq 0$ for all $W_1, W_2, x, y$ to establish convexity of the objective. But this monotonicity is clear by inspection: indeed if $x_l \geq 0$ then $(Wx)_k$ is increasing with $w_{kl}$ so the product is also increasing (note $e^{z}/(1 + e^z)$ is monotone increasing for all $z \in \mathbb{R}$) and similarly if $x_l <0$ then $(Wx)_k$ is decreasing, so the product is once again increasing giving monotonicity.

Since all $\{x_l\}_{l = 0}^{783}$ and $\{y_j\}_{j=0}^9$ lie on the interval $[0, 1]$ we can further conclude that $\sup_{x, y, W} || \partial_{w_{kl}} f ||_{\infty} \leq 2$, and hence for our objective function with the $\alpha$ regularisation term, we have an upper bound on the Lipshitz constant of the gradient as $ 7840 \times 2 + \alpha$, by considering
$$
\sup_{W, x, y} ||\nabla_W f(W, x, y) + \alpha W ||_{2} \leq \sum_{l = 0}^{783} \sum_{j=0}^9 \sup_{W, x, y}| \partial_{w_{jl}} f(W, x, y)| + \alpha \leq 15680 + \alpha
$$

This is of course far from optimal as you should not expect all the above triangle inequalities to be sharp at the same time, but computing analytically this Lipshitz constant is challenging, and in our numerical implementations we used learning rates larger than those theoretically guaranteed by this coarse upper bound.

### Implementation


We Implemented X different optimisation algorithms: Stochastic (Batch) Gradient Descent (SGD), Adam, [Shampoo](https://arxiv.org/abs/1802.09568) and (Stochastic) Coordinate Descent